# Constant Phase Shifter

## Definition

A constant phase shifter refers to a filter
that applies a frequency independent phase shift $\varphi$.
The transfer function $H_{\varphi}(\omega)$ reads

\begin{align}
H_{\varphi}(\omega) =
\begin{cases}
e^{+i \varphi}, & \omega > 0\\
e^{-i \varphi}, & \omega < 0\\
\cos\varphi, & \omega = 0
\end{cases}
\end{align}

* Keywords: fractional Hilbert transform, phase shifter, all-pass filter

* Applications

  - Discrete Hilbert transform filter ($\varphi = -\tfrac{\pi}{2}$),
    also called Hilbert transformer or $90$-degree phase shifter [Oppenheim, Sec. 11.4, p. 790].
    Due to this relation, constant phase shifter is also called fractional Hilbert transformer.

  - WFS pre-equalization filters (phase response only):
    2.5D ($\varphi = \tfrac{\pi}{4}$) and 3D ($\varphi = \frac{\pi}{2}$) configurations

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import util

def tf_constant_phase(omega, phase_angle):
    H = np.exp(1j * phase_angle * np.sign(omega))
    H[omega == 0] = np.cos(phase_angle)
    return H

phase_angle = np.pi / 2
wmin, wmax, wnum = -10, 10, 1000
omega = np.linspace(wmin, wmax, num=wnum)
H = tf_constant_phase(omega, phase_angle)

fig, axes = plt.subplots(figsize=(12, 3), ncols=2, gridspec_kw={'wspace':0.05})

axes[0].plot(omega / np.pi, np.abs(H))
axes[0].set_ylim(-1.1, 1.1)
axes[0].set_ylabel('Magnitude / 1')
axes[0].set_title('Magnitude Response')

axes[1].plot(omega / np.pi, np.angle(H) / np.pi)
axes[1].yaxis.tick_right()
axes[1].yaxis.set_label_position("right")
axes[1].set_ylim(-1, 1)
axes[1].set_ylabel(r'Phase / $\pi$')
axes[1].set_title('Phase Response');

for ax in axes:
    ax.set_xlim(wmin / np.pi, wmax / np.pi)
    ax.set_xlabel(r'$\omega$ / $\pi$')
    ax.grid()

## Continuous Time Impulse Response

The requirement for the constant phase shifter can be relaxed
so that the filter exhibits the desired phase response
only within a limited bandwidth.
The bandwidth of $H_{\varphi}(\omega)$ is limited to $\omega_c$,

\begin{align}
H_{\varphi}(\omega) =
\begin{cases}
e^{+i \varphi}, & 0 < \omega \le \omega_c\\
e^{-i \varphi}, & -\omega_c \le \omega < 0\\
\cos\varphi, & \omega = 0\\
0, & |\omega| > \omega_c
\end{cases}
\end{align}

The corresponding impulse response can be obtained
by evaluating the inverse Fourier transform,

\begin{align}
h_{\varphi}(t)
=&
\frac{1}{2 \pi} \int_{-\infty}^{\infty}
H_{\varphi}(\omega) e^{i \omega t} \text{d} \omega\\
=&
\frac{1}{2 \pi} \int_{-\omega_c}^{0}
e^{-i \varphi} e^{i \omega t} \text{d} \omega
+ \frac{1}{2 \pi} \int_{0}^{\omega_c}
e^{+i \varphi} e^{i \omega t} \text{d} \omega\\
=&
\frac{e^{-i \varphi}}{2 \pi} \frac{1}{it}
\big(1 - e^{-i \omega_c t}\big)
+ \frac{e^{+i \varphi}}{2 \pi} \frac{1}{it}
\big(e^{i \omega_c t} - 1\big)\\
=&
\frac{1}{\pi t} e^{-i(\frac{\omega_c}{2} t + \varphi)} \sin(\tfrac{\omega_c}{2} t)
+ \frac{1}{\pi t} e^{+i(\frac{\omega_c}{2} t + \varphi)} \sin(\tfrac{\omega_c}{2} t)\\
=&
\frac{2}{\pi t} \sin(\tfrac{\omega_c}{2} t) \cos(\tfrac{\omega_c}{2}t + \varphi)\\
=&
\frac{1}{\pi t} \big[\sin(\omega_c t + \varphi) - \sin\varphi\big]
\end{align}

where the addition theorem of trigonometric functions
is used in the last equality.
The above expression holds for $t \neq 0$.
The inverse Fourier transform for $t = 0$ reads

\begin{align}
h_{\varphi}(0)
=&
\frac{\omega_c}{2 \pi} e^{-i \varphi}
+ \frac{\omega_c}{2 \pi} e^{+i \varphi}\\
=& \frac{\omega_c}{\pi} \cos\varphi,
\end{align}

and thus

\begin{align}
h_{\varphi}(t) =&
\begin{cases}
\frac{1}{\pi t} \big[\sin(\omega_c t + \varphi) - \sin\varphi\big],
& t \neq 0\\
\frac{\omega_c}{\pi} \cos\varphi, & t = 0
\end{cases}
\end{align}

In [ ]:
def ir_constant_phase(time, phase_angle, bandwidth):
    omegac = 2 * np.pi * bandwidth
    h = np.zeros_like(time)
    idx_zero = (time == 0)
    if any(idx_zero):
        h[idx_zero] = omegac / np.pi * np.cos(phase_angle)
    h[~idx_zero] = 1 / np.pi / time[~idx_zero] \
                   * (np.sin(omegac * time[~idx_zero] + phase_angle) - np.sin(phase_angle))
    return h

samplerate = 20000
tmax = 0.01
tlength = int(2 * tmax * samplerate) + 1
t = np.arange(tlength) / samplerate - tmax

phase_angles = np.linspace(0, np.pi, num=5)
bandwidth = 1000

fig, ax = plt.subplots()
voffset = -2000
for i, phi in enumerate(phase_angles):
    h = ir_constant_phase(t, phi, bandwidth)
    plt.plot(t * 1000, h + i * voffset , label='{:0.1f}'.format(phi / np.pi))
plt.xlabel('$t$ / ms')
plt.xlim(-tmax * 1000, tmax * 1000)
plt.grid()
plt.legend(title='$\phi / \pi$', loc='upper right')
plt.title(r'Band Limited Constant Phase Shifters');

### Speical Case I ($\varphi=0$): Ideal Low Pass Filter

The 0-phase shifter constitutes an ideal low pass filter,

\begin{align}
H_{0}(\omega) =
\begin{cases}
1, & |\omega| \le \omega_c\\
0, & |\omega| > \omega_c.
\end{cases}
\end{align}

If the phase angle is set to $\varphi = 0$,
the impulse response is indeed a sinc function,

\begin{align}
h_{0}(t) =&
\begin{cases}
\frac{\omega_c}{\pi} \frac{\sin(\omega_c t)}{\omega_c t},
& t \neq 0\\
\frac{\omega_c}{\pi}, & t = 0
\end{cases}
\end{align}

In [ ]:
from scipy.special import sinc

omegac = 2 * np.pi * bandwidth
h_sinc = omegac / np.pi * sinc(omegac / np.pi * t)
h_zero_phase = ir_constant_phase(t, 0, bandwidth)

plt.figure()
plt.plot(t * 1000, h_zero_phase, label=r'$\varphi=0$')
plt.plot(t * 1000, h_sinc, label='sinc', color='k', linestyle='--')
plt.xlabel('$t$ / ms')
plt.xlim(-tmax * 1000, tmax * 1000)
plt.grid()
plt.legend()
plt.title('Band Limited 0 Phase Shifter');

### Special Case II ($\varphi=-\tfrac{\pi}{2}$): Hilbert Transform

The impulse response of a (band-limited) Hilbert transformer is obtained by setting $\varphi = -\frac{\pi}{2}$,

\begin{align}
h_\mathcal{H}(t) =
\begin{cases}
\frac{2}{\pi t} \sin^{2}(\tfrac{\omega_{s}}{2}t),& t \neq 0\\
0,& t = 0.
\end{cases}
\end{align}

In [ ]:
h_zero_phase = ir_constant_phase(t, 0, bandwidth)
h_hilbert = ir_constant_phase(t, -np.pi / 2, bandwidth)

plt.figure()
plt.plot(t * 1000, h_zero_phase, label='0')
plt.plot(t * 1000, h_hilbert, label='0.5 (Hilbert)')
plt.xlabel('$t$ / ms')
plt.xlim(-tmax * 1000, tmax * 1000)
plt.grid()
plt.legend(title=r'$\varphi$ / $\pi$');

The two phase shifters $h_{0}(t)$ (sinc function) and
$h_\mathcal{H}(t)$ (Hilbert transformer) form an analytic signal,

\begin{align}
\hat{h}(t) = \frac{1}{2}\big[h_{0}(t) + i h_\mathcal{H}(t)\big]
\end{align}

whose spectrum is a one-sided rectangle,

\begin{align}
\hat{H}(\omega) =
\begin{cases}
1,& 0 \le \omega \le \omega_c\\
0,& \text{elsewhere}.
\end{cases}
\end{align}

### Some Properties

From the definition of $H_{\varphi}(\omega)$, it follows that

* $h_{\varphi + 2 \pi}(t) = h_{\varphi}(t)$
* $h_{-\varphi}(t) = h_{\varphi}(-t)$
* $h_{\varphi + \pi} = -h_{\varphi}(t)$
* $h_{\pi - \varphi} = -h_{\varphi}(-t)$
* $h_{\varphi + \vartheta} = h_{\varphi}(t) \ast h_{\vartheta}(t)$,
  with $\ast$ denoting the convolution.

The symmetries (except the last one) are demonstrated in the following example. 

In [ ]:
Phase_Angles = np.linspace(0, 2 * np.pi, num=9)
voffset = 2000

plt.figure(figsize=(6, 8))
for i, phase_angle in enumerate(Phase_Angles):
    h = ir_constant_phase(t, phase_angle, bandwidth)
    plt.plot(t * 1000, h - i * voffset)
    plt.text(5, 200 - i * voffset,
             r'$\varphi = {:0.3f} \pi$'.format(phase_angle / np.pi))
plt.xlabel('$t$ / ms')
plt.xlim(-tmax * 1000, tmax * 1000)
plt.grid()

### Temporal Bandwidth

The impulse responses for different bandwidths are shown below.

In [ ]:
Bandwidths = 500, 1000, 2000, 5000
phase_angle = 0.25 * np.pi

fig, axes = plt.subplots(figsize=(12, 6), ncols=2, nrows=2, sharex=True, sharey=True)
for i, ax in enumerate(axes.flat):
    bandwidth = Bandwidths[i]
    h = ir_constant_phase(t, phase_angle, bandwidth)
    ax.plot(t * 1000, h, label='{}'.format(bandwidth))
    ax.set_xlim(-tmax * 1000, tmax * 1000)
    ax.grid()
    ax.set_title('Bandwidth = {}'.format(bandwidth))
    if i in [2, 3]:
        ax.set_xlabel('$t$ / ms')

## Digital Filter Design

### Time Descretization

Since $H(\omega)$ is band limited,
an aliasing-free sampling of the signal can be performed
provided that the sampling frequency $f_s$ satisfies
$2 \pi f_s \ge \omega_c$.
For $f_s = \frac{\omega_c}{2 \pi}$,
the discretized impulse response $h[n]$ reads
\begin{align}
h[n]
=& \frac{1}{f_s} \cdot h(\tfrac{n}{f_s})\\
=& \frac{1}{n \pi} \Big( \sin(\pi n + \varphi) - \sin\varphi \Big)\\
=&
\begin{cases}
-\frac{2}{n \pi} \sin\varphi, & \text{$n$: even}\\
0, & \text{$n$: odd}
\end{cases}
\end{align}
for $n \neq 0$ and
\begin{align}
h[0] = \cos\varphi,
\end{align}
therfore,
\begin{align}
h[n] =
\begin{cases}
\cos\varphi, & n = 0\\
-\frac{2}{n \pi} \sin\varphi, & \text{$n$ is odd}\\
0, & \text{$n$ is even and $n \neq 0$.}
\end{cases}
\end{align}

This is the impulse response of the digital filter
fulfilling the above mentioned magnitude and phase specifications.

Since the impulse response has an infinite support in
negative and positive time axes, i.e. $n \to \pm \infty$,
it has to be truncated and time shifted to obtain a realizable filter.
The FIR filter then exhibits a constant group delay.

In [ ]:
def discrete_ir_constant_phase(n, phase_angle):
    idx_zero = (n == 0)
    idx_odd = ~idx_zero * (n%2 == 1)
    h = np.zeros(len(n))
    h[idx_zero] = np.cos(phase_angle)
    h[idx_odd] = -2 / np.pi / n[idx_odd] * np.sin(phase_angle)
    return h

phase_angle = 0.25 * np.pi
bandwidth = 1000
omegac = 2 * np.pi * bandwidth
tmax = 0.01

oversamplerate = 20000
tlength = 2 * int(tmax * oversamplerate) + 1
t = np.arange(tlength) / oversamplerate - tmax
hc = ir_constant_phase(t, phase_angle, bandwidth)

samplerate = 2 * bandwidth
nlength = 2 * int(tmax * samplerate) + 1
n = np.arange(nlength) - (nlength - 1) / 2
hd = discrete_ir_constant_phase(n, phase_angle)
hd *= samplerate

plt.figure()
plt.plot(t * 1000, hc, label='continuous')
plt.plot(n / samplerate * 1000, hd, 'o', ms=5, label='sampled')
plt.xlabel('$t$ / ms')
plt.xlim(-tmax * 1000, tmax * 1000)
plt.grid()
plt.legend();

### Window Method

The design of the digital filter design can be also carried out
by specifying its spectrum in the discrete-time Fourier transform
domain within $\Omega \in [-\pi, \pi]$,

\begin{align}
H(e^{i\Omega}) =
\begin{cases}
e^{+i\varphi}, & 0 < \Omega < \pi\\
e^{-i\varphi}, & -\pi < \Omega < 0\\
\cos\varphi, & \Omega = 0, \pi
\end{cases}
\end{align}

where $\varphi$ again denotes the phase angle.
The inverse discrete-time Fourier transform is
\begin{align}
h[n] &= 
\frac{1}{2 \pi} \int_{-\pi}^{\pi}
H(e^{i\Omega}) e^{i \Omega n} \text{d}\Omega\\
&=
\frac{1}{2 \pi}
\Big[
\int_{-\pi}^{0} e^{-i \varphi} e^{i \Omega n} \text{d}\Omega
+ \int_{0}^{\pi} e^{+i \varphi} e^{i \Omega n} \text{d}\Omega
\Big]
\end{align}
Therefore the impulse response is
\begin{align}
h[0] &=
\frac{1}{2 \pi}
\Big[e^{-i\varphi} \cdot \pi + e^{+i\varphi} \cdot \pi\Big] \\
&= \cos\varphi
\end{align}
for $n = 0$ and
\begin{align}
h[n] &=
\frac{1}{2 \pi}
\Big[e^{-i \varphi} \frac{1}{in} \big(1 - (-1)^{n}\big)
+ e^{+i \varphi} \frac{1}{in} \big((-1)^{n} - 1\big) \Big]\\
&= \frac{1}{n \pi} \big((-1)^{n} - 1\big)
\frac{e^{+i \varphi} - e^{-i \varphi}}{2 i}\\
&=
\begin{cases}
-\frac{2}{n \pi} \sin\varphi, & \text{$n$: odd}\\
0, & \text{$n$: even},
\end{cases}
\end{align}
and for $n \neq 0$.
Thus,
\begin{align}
h[n] =
\begin{cases}
\cos\varphi, & n = 0\\
-\frac{2}{n \pi} \sin\varphi, & \text{$n$: odd}\\
0, & \text{$n$: even and $n \neq 0$.}
\end{cases}
\end{align}

The impulse response is odd symmetric if $\cos\varphi = 0$,
i.e. $\varphi$ being an odd multiple of $\frac{\pi}{2}$.
Due to the anti-symmetry for $n \neq 0$,
the DC response is determined by $h[0]$,

\begin{align}
H(0)
&= \sum_{n=-\infty}^{\infty} h[n] \cdot e^{i0n}\\
&= h[0]\\
&= \cos\varphi
\end{align}

As mentioned earlier, only a finite coefficients
can be used in practical usage.
It is a natural choice to truncated the coefficients symmetric
with respect to $n=0$.

\begin{align}
h_\text{d}[n] =
\begin{cases}
h[n-M],& n=[0, N-1]\\
0,& \text{elsewhere},
\end{cases}
\end{align}

where $N$ is the length of the digital filter,
and $M = \frac{N - 1}{2}$ the half length (for even $N$).
Notice that a time shift of $M$ is applied to make it causal.
The order of the FIR filter is $N - 1$.
Due to the symmetry for $n\neq 0$, the DC gain is still $\cos\varphi$.

In [ ]:
from scipy.signal import freqz, kaiser
from sfs.util import db

def constant_phase_shifter(filter_order, phase_angle, beta=0, frac_delay=0):
    filter_length = filter_order + 1
    n = np.arange(-filter_order / 2, filter_order / 2 + 1) - frac_delay
    h = discrete_ir_constant_phase(n, phase_angle)
    h *= kaiser(filter_length, beta=beta)
    return n, h

fs = 44100
fmin, fmax, fnum = 0, fs / 2, 1000
f = np.linspace(fmin, fmax, num=fnum)
omega = 2 * np.pi * f

phase_angle = -0.25 * np.pi
filter_order = 32
half_length = filter_order / 2
group_delay = half_length / fs

n, h = constant_phase_shifter(filter_order, phase_angle)
_, H = freqz(h, 1, f, fs=fs)
H *= np.exp(1j * omega * group_delay)
Hphase = np.mod(np.angle(H), 2 * np.pi)

fig, ax = plt.subplots(ncols=2, figsize=(15, 4))

ax[0].stem(n, h)
ax[0].set_xlabel('$n$ / sample')
ax[0].set_ylabel('$h[n]$')
ax[0].set_xlim(-half_length, half_length)
ax[0].grid()
ax[0].set_title(r'Impulse Response ($\varphi={:0.2f} \pi$)'.format(phase_angle / np.pi))

ax[1].plot(f, db(H), label='Magnitude')
ax[1].set_xlim(fmin, fmax)
ax[1].set_ylim(-6, 6)
ax[1].set_xlabel('$f$ / Hz')
ax[1].set_ylabel('Magnitude / dB')
ax[1].grid()
ax[1].legend(loc='lower right')
ax[1].set_title(r'Frequency Response ($\varphi={:0.2f} \pi$)'.format(phase_angle / np.pi))

ax_phase = ax[1].twinx()
ax_phase.plot(f, Hphase / np.pi, color='red', label='Phase')
ax_phase.set_ylabel('Phase / $\pi$', color='red')
ax_phase.set_ylim(0, 2)
ax_phase.legend(loc='upper right');

It can be seen that the gain at the baseband
and the Nyquist frequency is generally not unity
and depends on the phase angle $\varphi$.
The truncation of the impulse response causes
ripples in the magnitude and phase responses
as depicted in the right figure.

### Tapering

In [ ]:
Betas = 0, 2, 6

fig, axes = plt.subplots(figsize=(15, 4), ncols=2, sharex=True)

for beta in Betas:
    n, h = constant_phase_shifter(filter_order, phase_angle, beta=beta)
    _, H = freqz(h, 1, f, fs=fs)
    H *= np.exp(1j * omega * group_delay)  # remove group delay
    axes[0].plot(f, db(H), label='{}'.format(beta))
    axes[1].plot(f, np.angle(H) / np.pi, label='{}'.format(beta))

for ax in axes:
    ax.set_xlabel('$f$ / Hz')
    ax.set_xlim(fmin, fmax)
    ax.legend(title=r'$\beta$', loc=1)
    ax.grid()

axes[0].set_ylabel('Magnitude / dB')
axes[0].set_ylim(-6, 6)
axes[0].set_title('Magnitude Responses')

axes[1].plot(0, phase_angle / np.pi, '<')
axes[1].set_ylabel('Phase / $\pi$')
axes[1].set_ylim(-1, 1)
axes[1].set_title('Phase Responses');

The impulse response is tapered by using 
Kaiser windows with different values of $\beta$,
where $\beta = 0$ constitutes a rectangular window
and $\beta = 6$ a Hann window.
The ripples are suppressed to some extent,
which comes at the expanse of a narrower passband.
The (non-ideal) gain at $f=0,\tfrac{f_s}{2}$, however,
remains.

### Filter Length

In [ ]:
Orders = 128, 256, 512, 1024
phase_angle = -0.25 * np.pi
phase_angle_deg = np.rad2deg(phase_angle)
beta = 6

fmin, fmax, fnum = 2, fs / 2, 3000
f = np.logspace(np.log10(fmin), np.log10(fmax), num=fnum)
omega = 2 * np.pi * f

fig, axes = plt.subplots(figsize=(12, 8), ncols=2, nrows=2, sharex=True, sharey='row',
                        gridspec_kw={'wspace':0.05, 'hspace':0.05})
voffset = 6

for i, filter_order in enumerate(Orders):
    group_delay = filter_order / 2 / fs
    n, h = constant_phase_shifter(filter_order, phase_angle)
    n, hwin = constant_phase_shifter(filter_order, phase_angle, beta=beta)
    _, H = freqz(h, 1, f, fs=fs)
    _, Hwin = freqz(hwin, 1, f, fs=fs)
    phase_shift = np.exp(1j * omega * group_delay)
    H *= phase_shift
    Hwin *= phase_shift

    axes[0, 0].semilogx(f, db(H) - i * voffset, label='{}'.format(filter_order + 1))
    axes[0, 1].semilogx(f, db(Hwin) - i * voffset)
    axes[1, 0].semilogx(f, np.angle(H) / np.pi)
    axes[1, 1].semilogx(f, np.angle(Hwin) / np.pi)

for ax in axes.flatten():
    ax.grid()

axes[0, 0].set_xlim(fmin, fmax)
axes[0, 0].set_ylim(-22, 3)
axes[0, 0].set_yticks(np.arange(-24, 12, 6))
axes[0, 0].set_ylabel('Magnitude / dB')
axes[0, 0].legend(title='Filter length')
axes[0, 0].set_title('No windowing')
axes[0, 1].set_title(r'Kaiser window $\beta$={}'.format(beta))
axes[1, 0].set_ylim(-1, 1)
axes[1, 0].set_xlabel('$f$ / Hz')
axes[1, 0].set_ylabel('Phase / $\pi$')
axes[1, 1].set_xlabel('$f$ / Hz')
fig.suptitle(r'$\varphi = {} \pi$'.format(phase_angle / np.pi));

Note that the frequency responses (both magnitude and phase)
are now depicted in logarithmic frequency axes.
As the filter length increases the roll-off occurs at lower frequencies.
This is a typical property of FIR filters
where the low frequency resolution is limited by its length.
Both the magnitude and phase responses can be smoothed
applying a window to the FIR coefficients.

### Magnitude and Phase Deviations

In [ ]:
num_phase = 8
phase_angles = np.linspace(-np.pi, 0, num=num_phase, endpoint=False)
filter_order = 128
half_length = filter_order / 2
group_delay = half_length / fs

fmin, fmax, fnum = 20, fs / 2, 1000
f = np.logspace(np.log10(fmin), np.log10(fmax), num=fnum)
omega = 2 * np.pi * f
voffset_ir = 2
voffset_tf = 6

plt.figure(figsize=(15, 10))
ax1 = plt.subplot2grid((2, 2), (0, 0), rowspan=2)
ax2 = plt.subplot2grid((2, 2), (0, 1))
ax3 = plt.subplot2grid((2, 2), (1, 1))
for i, phi in enumerate(phase_angles):
    n, h = constant_phase_shifter(filter_order, phi)
    _, H = freqz(h, 1, f, fs=fs)
    H *= np.exp(1j * omega * group_delay)

    ax1.plot(n, h - i * voffset_ir, label='{:0.3f}'.format(phi / np.pi))
    ax2.semilogx(f, db(H) - i * voffset_tf)
    ax2.plot(fmin + 2, db(np.cos(phi)) - i * voffset_tf, 'k<')
    ax3.semilogx(f, np.mod(np.unwrap(np.angle(H)), 2 * np.pi) / np.pi)
    ax3.plot(fmax / 2, np.mod(phi, 2 * np.pi) / np.pi, 'kx')
    
ax1.set_xlim(-half_length, half_length)
ax1.set_yticks(np.arange(-num_phase * voffset_ir, 2, 2))
ax1.grid()
ax1.set_xlabel('$n$ / sample')
ax1.legend(title='Phase angle', loc='upper right')
ax1.set_title('Impulse Responses')
ax2.set_yticks(np.arange(-num_phase * voffset_tf, 12, 6))
ax2.set_xlim(fmin, fmax)
ax2.set_ylim(-num_phase * voffset_tf, 3)
ax2.grid()
ax2.set_ylabel('Magnitude / dB')
ax2.set_title('Magnitude Responses')
ax3.set_xlim(fmin, fmax)
ax3.set_ylim(0, 2)
ax3.grid()
ax3.set_xlabel('$f$ / Hz')
ax3.set_ylabel(r'Phase / $\pi$')
ax3.set_title('Phase Responses');

In the top right figure,
the DC response $H(0) = \cos \varphi$ is indicated by $\blacktriangleleft$.
In the bottom right, the target phase is indicated by triangles $\times$.

### Phase Equalization

In [ ]:
from scipy.signal import fftconvolve as conv

filter_order = 1024
half_length = filter_order / 2
group_delay = half_length / fs
phi = np.pi / 4

n, hf = constant_phase_shifter(filter_order, phi, beta=6)
n, hb = constant_phase_shifter(filter_order, -phi, beta=6)
h = conv(hf, hb)
h *= 1 / np.linalg.norm(h)

fmin, fmax, fnum = 2, fs / 2, 1000
f = np.logspace(np.log10(fmin), np.log10(fmax), num=fnum)
omega = 2 * np.pi * f
_, Hf = freqz(hf, 1, f, fs=fs)
_, Hb = freqz(hb, 1, f, fs=fs)
_, H = freqz(h, 1, f, fs=fs)

Hf *= np.exp(1j * omega * group_delay)
Hb *= np.exp(1j * omega * group_delay)
H *= np.exp(1j * omega * 2 * group_delay)

fig, axes = plt.subplots(figsize=(10, 4), ncols=2, gridspec_kw={'wspace':0.05})

axes[0].semilogx(f, db(Hf), 'b', alpha=0.5, label=r'$+\phi$')
axes[0].semilogx(f, db(Hb), 'b--', alpha=0.5, label=r'$-\phi$')
axes[0].semilogx(f, db(H), 'k', alpha=0.5, label=r'Equalized')
axes[0].semilogx(fmin, db(np.cos(phi)), 'k<')
axes[0].set_ylim(-9, 9)
axes[0].set_ylabel('$|H(\omega)|$ / dB')
axes[0].legend()

axes[1].semilogx(f, np.unwrap(np.angle(Hf)) / np.pi, 'b', alpha=0.5, label=r'$+\phi$')
axes[1].semilogx(f, np.unwrap(np.angle(Hb)) / np.pi, 'b--', alpha=0.5, label=r'$-\phi$')
axes[1].semilogx(f, np.unwrap(np.angle(H)) / np.pi, 'k', alpha=0.5, label=r'Equalized')
axes[1].yaxis.tick_right()
axes[1].yaxis.set_label_position("right")
axes[1].set_ylabel(r'$\angle H(\omega)$ $/\pi$')

for ax in axes:
    ax.set_xlim(fmin, fmax)
    ax.set_xlabel('$f$ / Hz')
    ax.grid()

fig.suptitle(r'Phase Equalization ($\varphi = \pm{} \pi$)'.format(phi / np.pi));

In this example, the response of a phase shifter ($\phi$, solid blue) is
equalized by using an opposite phase shifter ($-\phi$, dashed blue).
As can be seen in the right figure,
the phase is perfectly corrected (gray).
However, since the magnitude response of both phase shifters exhibit
a roll-off around $f=0, \tfrac{f_s}{2}$,
a doubled attenuation is observed in the equalized response.
The triangle indicates the DC gain $\cos\phi$.

## Fractional Pre-Delay

An arbitrary delay $\tau$ can be applied to the analytic expression
of the constant phase shifter

\begin{align}
h_{\varphi}(t - \tau) =&
\begin{cases}
\frac{1}{\pi (t - \tau)} \big[\sin(\omega_c (t - \tau) + \varphi) - \sin\varphi\big],
& t \neq \tau\\
\frac{\omega_c}{\pi} \cos\varphi, & t = \tau
\end{cases}
\end{align}

and discretized in time,

\begin{align}
h_{\varphi}^\tau[n] =&
\begin{cases}
\frac{1}{\pi (n - f_s \tau)} \big[\sin(\pi (n - f_s\tau) + \varphi) - \sin\varphi\big],
& n \neq \tau f_s\\
\cos\varphi, & n = \tau f_s
\end{cases}
\end{align}

In the following example, digital constant phase shifters are designed
for different values of $\tau$.
The coefficients are tapered by an asymmetric Kaiser window function.

In [ ]:
def asymmetric_kaiser(N, frac_delay, beta):
    n0 = np.round(frac_delay)
    hleft = kaiser(2 * (N / 2 + n0) + 1, beta=beta)[:int(N / 2 + n0 + 1)]
    hright = kaiser(2 * (N / 2 - n0) + 1, beta=beta)[int(N / 2 - n0 + 1):]
    return np.concatenate((hleft, hright))

def delayed_constant_phase_shifter(filter_order, phase_angle, fs, beta=0, frac_delay=0):
    filter_length = filter_order + 1
    n = np.arange(-filter_order / 2, filter_order / 2 + 1) - frac_delay
    h = discrete_ir_constant_phase(n, phase_angle)
    h = ir_constant_phase(n / fs, phase_angle, bandwidth=fs / 2)
    h *= asymmetric_kaiser(filter_order, frac_delay, beta=beta)
    return n, h

In [ ]:
from matplotlib import cm
colors = cm.viridis.colors

phase_angle = np.pi / 4
filter_order = 64
half_length = filter_order / 2
beta = 8.6
fractional_delays = np.linspace(-10, 10, num=15, endpoint=True)
n = np.arange(-filter_order / 2, filter_order / 2 + 1)

fmin, fmax, fnum = 20, fs / 2, 1000
f = np.logspace(np.log10(fmin), np.log10(fmax), num=fnum)
omega = 2 * np.pi * f

fig, axes = plt.subplots(figsize=(15, 8), ncols=3)
viridis = cm.get_cmap('viridis', len(fractional_delays)).colors
voffset_ir = 1
voffset_tf = 0

for i, fd in enumerate(fractional_delays):
    _, h = delayed_constant_phase_shifter(filter_order, phase_angle, fs=fs, beta=beta, frac_delay=fd)
    w, H = freqz(h / fs, 1, f, fs=fs)
    group_delay = (half_length + fd) / fs
    H *= np.exp(1j * omega * group_delay)

    axes[0].plot(n, h / fs - i * voffset_ir, color=viridis[i], label='{:0.1f}'.format(fd))
    axes[0].set_xlabel('$n$')

    axes[1].semilogx(f, db(H) - i * voffset_tf, color=viridis[i])
    axes[1].semilogx(fmin, db(np.cos(phase_angle)) - i * voffset_tf, 'k<')
    axes[2].semilogx(f, np.unwrap(np.angle(H)))
    axes[2].semilogx(fmax, phase_angle, 'k<')
    axes[0].text(-20, - i * voffset_ir, '{:0.1f}'.format(fd))

for ax in axes:
    ax.grid()
axes[0].set_xlim(-50, 50)
axes[1].set_ylim(-len(fractional_delays) * voffset_tf - 6, 3);

The deviations in the lower frequencies varies with $\tau$.
Positive pre-delay (more pre-delay) tends to
moderates the low frequency attenuation.
The variations in the phase response are marginal.

## Decomposition

The constant phase shifter can be decomposed into
real and imaginary components by exploiting the Euler's formula,

\begin{align}
H(i\omega) &=
\begin{cases}
\cos\varphi + i\sin\varphi,& \omega > 0\\
\cos\varphi + i\sin\varphi,& \omega < 0
\end{cases}
\end{align}

therefore it consists of two systems

\begin{align}
H(i\omega) = \cos\varphi H_{0}(i\omega) + \sin\varphi H_{\pi/2}(i\omega)
\end{align}

where $H_{0}(i\omega) = 1$ and

\begin{align}
H_{\pi/2}(i\omega) =
\begin{cases}
i,& \omega > 0\\
-i,& \omega <0.
\end{cases}
\end{align}

Alternatively, the Hilbert transformer can be used
to describe the imaginary part,

\begin{align}
H(i\omega) = \cos\varphi H_{0}(i\omega) - \sin\varphi H_\mathcal{H}(i\omega).
\end{align}

This shows that any constant phase shifter can be considered
as a linear combination of an ideal system $H_{0}(i\omega)$
and a $\tfrac{\pi}{2}$-phase shifter $H_{\pi/2}(i\omega)$
or a Hilbert transformer $H_\mathcal{H}(i\omega)$.